## ASD Cohort Selection
To obtain our sample cohort of patients diagnosed with ASD, we queried the insurance database of member-linked Icd codes. 

We create table (ASDFacts) where:
- the rows include every insurance database member with ICD-9 and/or ICD-10 codes associated with ASD
- the columns include member ID, date service started, and Icd code

### Load the R libraries

In [ ]:
library("devtools")
library("SqlServerJtds")
library("SqlTools")
library("FactToCube")

### Set up the connection to the DB server

In [ ]:
cn = connect.sql.server(
    database="XXXX",
    domain="YYYY",
    server.address="ZZZZ",
    user= yourUserName,
    password= yourPassword)
cn

### ASD patient selection

In [ ]:
dbSendUpdate( cn, "SELECT *
           INTO ASDFacts
           FROM 
           FactIcd F
           WHERE 
           F.Icd in ('299.0', '299.00', '299.01', '299.8', '299.80', '299.81', '299.9', '299.90', '299.91',
                     '299', '299.1', '299.10', '299.11', 'F84.', 'F84.0', 'F84.3', 'F84.5', 'F84.9')")

Create table (ASDMembersAtLeast1) where:
- the rows include every member
- the columns include the member identifier, date service started, gender, & birth year for all members with at least one diagnosis of ASD

Age range was restricted to 0 - 100 years old to filter out invalid birth years

In [ ]:
dbSendUpdate( cn, "SELECT 
           A.MemberId, DateServiceStarted, Gender, BirthYear,( YEAR ( DateServiceStarted ) - BirthYear ) AS Age 
           INTO ASDMembersAtLeast1
           FROM ASDFacts A
           INNER JOIN Members M 
           ON A.MemberId = M.MemberId
           WHERE
           ( YEAR ( DateServiceStarted ) - BirthYear ) >= 0
           AND ( YEAR ( DateServiceStarted ) - BirthYear ) <= 100
           GROUP BY A.MemberId, DateServiceStarted")

Create new table (ASDMembersAtLeast3) that includes only members diagnosed with ASD ICD-9 or ICD-10 codes on 3 different dates.


In [ ]:
dbSendUpdate( cn, "SELECT 
           A.MemberId, COUNT ( A.MemberId ) AS nASDCounts
           INTO ASDMembersAtLeast3
           FROM ASDMembersAtLeast1 A
           GROUP BY MemberId 
           HAVING COUNT ( A.MemberId ) >= 3")

Further refine ASD cohort table to only include the members having insurance coverage for at least 12 months:
- Resulting table (ASD12mo3dx) included member zip codes

In [ ]:
dbSendUpdate( cn, "SELECT E.MemberId, min( E.ZipCode ) AS ZipCode,
           COUNT ( DISTINCT E.EffectiveDate ) AS nMonthsObservation,
           BirthYear,Gender
           INTO ASD12mo3dx
           FROM Enrollment E
           INNER JOIN ASDMembersAtLeast1 I
           ON E.MemberId = I.MemberId
           INNER JOIN ASDMembersAtLeast3
           ON I.MemberId = ASDMembersAtLeast3.MemberId
           GROUP BY E.MemberId,I.BirthYear,I.Gender
           HAVING COUNT ( DISTINCT E.EffectiveDate ) >= 12")

Refine ASD cohort table to include the following information for each member diagnosed with ASD:
- Member ID
- Birth year
- Gender
- Zip code
- Date service started
- Age

In [ ]:
dbSendUpdate( cn, "SELECT ASD.MemberId, C.BirthYear, C.Gender, ZipCode, DateServiceStarted,
           ( YEAR ( ASD.DateServiceStarted ) - C.BirthYear ) AS Age
           INTO ASDMembersDiagnoses
           FROM ASDMembersAtLeast1 ASD 
           INNER JOIN ASD12mo3dx C ON
           ASD.MemberId = C.MemberId")

Final working ASD cohort table reporting:
- Member ID
- Gender
- Birth year
- The member’s age at 1st diagnosis (of anything),
- Number medical visits
- Zip codes to use for demographic analysis

In [ ]:
dbSendUpdate( cn, "SELECT MemberID, Gender, BirthYear,MIN (Age) AS FirstDiagnosedAge,COUNT (*) AS NDiagnoses, 
           COUNT (DISTINCT DateServiceStarted) AS NVisits, MIN (ZipCode) AS ZipCode
           INTO ASDMembers
           FROM ASDMembersDiagnoses
           GROUP BY MemberID, Gender, BirthYear")